In [1]:
import xarray as xr
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

%load_ext autoreload
%autoreload 2

### Paths input data

In [2]:
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, RESULTS_DIR

# PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
# PATH_DT = OUTPUT_DATA_DIR / '/dT_data_rcmip_models.nc'
PATH_DT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


#### Uncertainty data from Chris

In [3]:
PATH_DT_UNCERTAINTY = OUTPUT_DATA_DIR / 'dT_uncertainty_data_FaIR_chris_ed02-3.nc'

## Set values:

In [4]:
first_y = 1750
last_y = 2100

**Set reference year for temperature change:**

In [5]:
ref_year = 2020

In [6]:
FIGURE_DIR = RESULTS_DIR / 'figures_recommendation/'

TABLE_DIR = RESULTS_DIR / 'tables_recommendation/'

In [7]:
from pathlib import Path
Path(FIGURE_DIR).mkdir(parents=True, exist_ok=True)
Path(TABLE_DIR).mkdir(parents=True, exist_ok=True)

In [8]:
percentile = 'percentile'
climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
name_deltaT = 'Delta T'


### Define variables to look at:

In [9]:
# variables to plot:
variables_erf_comp = [
    'ch4',
    'aerosol-total-with_bc-snow',
    #'aerosol-radiation_interactions',
    #'aerosol-cloud_interactions',
    'aerosol-total',
    'o3',
    'HFCs',
    # 'F-Gases|HFC',
    'bc_on_snow',
    'total_anthropogenic',
    #'total',
]
variables_in_sum = [
    'aerosol-total-with_bc-snow',
    'ch4',
    # 'aerosol-radiation_interactions',
    # 'aerosol-cloud_interactions',
    #'aerosol-total',
    'o3',
    'HFCs',
    #'bc_on_snow'
]

# total ERFs for anthropogenic and total:
#variables_erf_tot = ['total_anthropogenic',
#                     'total']

scenarios_fl_370 = ['ssp370', 'ssp370-lowNTCF-aerchemmip', 'ssp370-lowNTCF-gidden'  # Due to mistake here
                    ]

### Scenarios:

In [10]:
scenarios_fl = ['ssp119',
                'ssp126',
                'ssp245',
                'ssp370',
                'ssp370-lowNTCF-aerchemmip',
                #'ssp370-lowNTCF-gidden',
                'ssp370-lowNTCF-gidden',
                'ssp585'
               ]

In [11]:
median = 'median'
perc5 = '5th percentile'
perc95 = '95th percentile'
recommendation = 'recommendation'

## Open dataset:

### Integrate:
The code below opens the file generated in [2_compute_delta_T.ipynb](2_compute_delta_T.ipynb) by integrating

\begin{align*} 
\Delta T (t) &= \int_0^t ERF(t') IRF(t-t') dt' \\
\end{align*}

where IRF is the impulse response function and ERF is the effective radiative forcing from RCMIP. 

In [12]:
ds_DT = xr.open_dataset(PATH_DT)
ds_uncertainty = xr.open_dataset(PATH_DT_UNCERTAINTY)

In [13]:
for var in variables_erf_comp:
    da5 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p05-p50']    
    da95 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p95-p50']
    da5.plot(label=var)
    da95.plot(label=var)

In [14]:
ds_uncertainty.variable

<xarray.DataArray 'variable' (variable: 8)>
array(['HFCs', 'Sum SLCF (Methane, Aerosols, Ozone, HFCs)', 'aerosol-total',
       'bc_on_snow', 'ch4', 'o3', 'total_anthropogenic',
       'aerosol-total-with_bc-snow'], dtype=object)
Coordinates:
  * variable  (variable) object 'HFCs' ... 'aerosol-total-with_bc-snow'

In [15]:
v_sum = 'Sum SLCF (Methane, Aerosols, Ozone, HFCs)'
ds_uncertainty['p05-p50'].sel(year=2040, scenario='ssp119', variable=v_sum)#.variables#.plot()#, variable='%%SVG')%%SVG

<xarray.DataArray 'p05-p50' (base_period: 1)>
array([-0.126748])
Coordinates:
  * base_period  (base_period) int64 2020
    scenario     <U6 'ssp119'
    year         int64 2040
    variable     <U41 'Sum SLCF (Methane, Aerosols, Ozone, HFCs)'

In [16]:
ds_uncertainty.variable

<xarray.DataArray 'variable' (variable: 8)>
array(['HFCs', 'Sum SLCF (Methane, Aerosols, Ozone, HFCs)', 'aerosol-total',
       'bc_on_snow', 'ch4', 'o3', 'total_anthropogenic',
       'aerosol-total-with_bc-snow'], dtype=object)
Coordinates:
  * variable  (variable) object 'HFCs' ... 'aerosol-total-with_bc-snow'

In [17]:
ds_DT

<xarray.Dataset>
Dimensions:     (percentile: 1, scenario: 11, variable: 19, year: 351)
Coordinates:
  * year        (year) int64 1750 1751 1752 1753 1754 ... 2097 2098 2099 2100
  * variable    (variable) object 'co2' 'ch4' ... 'HFCs'
  * scenario    (scenario) object 'ssp534-over' ... 'ssp370-lowNTCF-gidden'
  * percentile  (percentile) object 'recommendation'
Data variables:
    ERF         (scenario, variable, year) float64 ...
    time        (year) datetime64[ns] ...
    delta_t     (year) float64 ...
    Delta T     (percentile, scenario, variable, year) float64 ...

## Merge uncertainty and original: 


In [18]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

<IPython.core.display.Javascript object>

hallo


### Add sum

In [19]:
_str = ''
_vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_in_sum]
for var in _vl: 
    _str += f'{var}, '

# ax.set_title('Temperature change, sum SLCF  (%s)' % _str[:-2])


vn_sum = 'Sum SLCF (%s)' % _str[:-2]
print(vn_sum)
#_st = vn_sum.replace('(','').replace(')','').replace(' ','_').replace(',','')+'.csv'

_da_sum  = ds_DT[name_deltaT].sel(variable=variables_in_sum).sum(variable)
#_da = ds_DT[name_deltaT].sel(variable=variables_erf_comp).sum(variable).sel(year=slice(int(s_y2), int(e_y2))) - ds_DT_sy
_da_sum#.assin_coord()
#_ds_check = ds_DT.copy()
ds_DT
#xr.concat([_ds_check[name_deltaT],_da_sum], dim=variable)

dd1=_da_sum.expand_dims(
    {'variable':
    [vn_sum]})
#dd1=dd1.to_dataset()

ds_DT = xr.merge([ds_DT,dd1])

Sum SLCF (Aerosols, Methane, Ozone, HFCs)


In [20]:
percentiles_to_keep = ['p05-p50','p16-p50','p84-p50','p95-p50']

In [21]:
_da = ds_uncertainty[percentiles_to_keep].to_array('percentile')
_ds = _da.rename(name_deltaT).to_dataset()

In [22]:
ds_DT= xr.concat([ds_DT[name_deltaT],_ds[name_deltaT]], dim='percentile').to_dataset()

In [23]:
ds_DT

<xarray.Dataset>
Dimensions:      (base_period: 1, percentile: 5, scenario: 11, variable: 21, year: 351)
Coordinates:
  * year         (year) int64 1750 1751 1752 1753 1754 ... 2097 2098 2099 2100
  * variable     (variable) object 'HFCs' ... 'volcanic'
  * scenario     (scenario) object 'ssp119' 'ssp126' ... 'ssp534-over' 'ssp585'
  * base_period  (base_period) int64 2020
  * percentile   (percentile) object 'recommendation' 'p05-p50' ... 'p95-p50'
Data variables:
    Delta T      (percentile, scenario, variable, year, base_period) float64 0.0 ... nan

In [24]:
ds_DT.sel(year=2040, scenario='ssp119', percentile='recommendation', variable=v_sum)#.variables#.plot()#, variable='%%SVG')%%SVG

<xarray.Dataset>
Dimensions:      (base_period: 1)
Coordinates:
    year         int64 2040
    variable     <U41 'Sum SLCF (Methane, Aerosols, Ozone, HFCs)'
    scenario     <U6 'ssp119'
  * base_period  (base_period) int64 2020
    percentile   <U14 'recommendation'
Data variables:
    Delta T      (base_period) float64 nan

## Make csv table of total values:

In [25]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

In [26]:
reference_year = ref_year
start_y_tabel = 2015
end_y_tabel = last_y

In [28]:
def get_fn(var_name, s_y, e_y, ref_y, perc):
    _st = var_name.replace('(','').replace(')','').replace(' ','_').replace(',','')#+'.csv'
    fn = f'{perc}_{_st}_{s_y}-{e_y}_refyear{ref_y}.csv'
    return fn

def make_sum_slcfs_tabel(sum_vn = vn_sum, percentile=recommendation):
    #_str = ''
    #_vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_erf_comp]
    #for var in _vl:
    #    _str += f'{var}, '
    #vn_sum = 'Sum SLCF (%s)' % _str[:-2]

    fn = get_fn(vn_sum, start_y_tabel, end_y_tabel, ref_year, percentile)



    # ref year value:
    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=sum_vn,
        year=reference_year
    ).squeeze()

    # all values from s_y to e_y
    _da = ds_DT[name_deltaT].sel(
        variable=sum_vn,
        year=slice(start_y_tabel, end_y_tabel)
    ) - ds_DT_sy

    # Choose recommendation::
    _pl_da = _da.sel(percentile=percentile).squeeze()
    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    fn = TABLE_DIR / fn
    print(percentile)
    display(df.loc[2040])
    df.to_csv(fn)
    
    
#for prc in [recommendation, 'p05-p50','p95-p50']:
#    make_sum_slcfs_tabel(percentile=prc)

In [29]:
import pandas as pd

## TABLE EACH VAR

In [30]:

def make_slcfs_tabel(var, percentile=recommendation):
    _str = ''
    _st = get_fn(var, start_y_tabel, end_y_tabel, ref_year, percentile)


    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=var, 
        year=reference_year,
    ).squeeze()
    
    _da = ds_DT[name_deltaT].sel(
        variable=var,
        year=slice(start_y_tabel, end_y_tabel)) - ds_DT_sy
    
    # Take recommendation::
    _pl_da = _da.sel(percentile=recommendation).squeeze()

    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    display(df)
    fn = TABLE_DIR/_st
    df.to_csv(fn)

for var in variables_erf_comp+[vn_sum]:
    for prc in [recommendation, 'p05-p50','p95-p50']:
        make_slcfs_tabel(var, percentile=prc)

scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.010446,-0.010175,-0.011257,-0.013001,-0.011757,-0.011757,-0.011543,-0.012349,-0.012349,-0.011188,-0.011061,recommendation
2016,-0.008579,-0.008308,-0.009391,-0.011135,-0.009890,-0.009890,-0.009677,-0.010482,-0.010482,-0.009322,-0.009194,recommendation
2017,-0.006735,-0.006464,-0.007546,-0.009290,-0.008046,-0.008046,-0.007832,-0.008638,-0.008638,-0.007477,-0.007350,recommendation
2018,-0.004837,-0.004566,-0.005649,-0.007393,-0.006148,-0.006148,-0.005935,-0.006740,-0.006740,-0.005580,-0.005452,recommendation
2019,-0.002899,-0.002628,-0.003710,-0.005454,-0.004210,-0.004210,-0.003996,-0.004802,-0.004802,-0.003641,-0.003514,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157716,-0.149590,0.000742,-0.006809,0.265116,0.265116,-0.124106,0.125782,0.178841,-0.161917,0.153667,recommendation
2097,-0.159108,-0.151092,0.000247,-0.006622,0.268031,0.268031,-0.123713,0.125689,0.178938,-0.163362,0.152101,recommendation
2098,-0.160467,-0.152549,-0.000213,-0.006431,0.270925,0.270925,-0.123291,0.125559,0.179021,-0.164730,0.150545,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.010446,-0.010175,-0.011257,-0.013001,-0.011757,-0.011757,-0.011543,-0.012349,-0.012349,-0.011188,-0.011061,p05-p50
2016,-0.008579,-0.008308,-0.009391,-0.011135,-0.009890,-0.009890,-0.009677,-0.010482,-0.010482,-0.009322,-0.009194,p05-p50
2017,-0.006735,-0.006464,-0.007546,-0.009290,-0.008046,-0.008046,-0.007832,-0.008638,-0.008638,-0.007477,-0.007350,p05-p50
2018,-0.004837,-0.004566,-0.005649,-0.007393,-0.006148,-0.006148,-0.005935,-0.006740,-0.006740,-0.005580,-0.005452,p05-p50
2019,-0.002899,-0.002628,-0.003710,-0.005454,-0.004210,-0.004210,-0.003996,-0.004802,-0.004802,-0.003641,-0.003514,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157716,-0.149590,0.000742,-0.006809,0.265116,0.265116,-0.124106,0.125782,0.178841,-0.161917,0.153667,p05-p50
2097,-0.159108,-0.151092,0.000247,-0.006622,0.268031,0.268031,-0.123713,0.125689,0.178938,-0.163362,0.152101,p05-p50
2098,-0.160467,-0.152549,-0.000213,-0.006431,0.270925,0.270925,-0.123291,0.125559,0.179021,-0.164730,0.150545,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.010446,-0.010175,-0.011257,-0.013001,-0.011757,-0.011757,-0.011543,-0.012349,-0.012349,-0.011188,-0.011061,p95-p50
2016,-0.008579,-0.008308,-0.009391,-0.011135,-0.009890,-0.009890,-0.009677,-0.010482,-0.010482,-0.009322,-0.009194,p95-p50
2017,-0.006735,-0.006464,-0.007546,-0.009290,-0.008046,-0.008046,-0.007832,-0.008638,-0.008638,-0.007477,-0.007350,p95-p50
2018,-0.004837,-0.004566,-0.005649,-0.007393,-0.006148,-0.006148,-0.005935,-0.006740,-0.006740,-0.005580,-0.005452,p95-p50
2019,-0.002899,-0.002628,-0.003710,-0.005454,-0.004210,-0.004210,-0.003996,-0.004802,-0.004802,-0.003641,-0.003514,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157716,-0.149590,0.000742,-0.006809,0.265116,0.265116,-0.124106,0.125782,0.178841,-0.161917,0.153667,p95-p50
2097,-0.159108,-0.151092,0.000247,-0.006622,0.268031,0.268031,-0.123713,0.125689,0.178938,-0.163362,0.152101,p95-p50
2098,-0.160467,-0.152549,-0.000213,-0.006431,0.270925,0.270925,-0.123291,0.125559,0.179021,-0.164730,0.150545,p95-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.061090,-0.060200,-0.056988,-0.023356,-0.039367,-0.045977,-0.045977,-0.049090,-0.049090,-0.065988,-0.065988,recommendation
2016,-0.045630,-0.044741,-0.041528,-0.007896,-0.023907,-0.030517,-0.030517,-0.033631,-0.033631,-0.050528,-0.050528,recommendation
2017,-0.031191,-0.030302,-0.027090,0.006542,-0.009469,-0.016079,-0.016079,-0.019192,-0.019192,-0.036090,-0.036090,recommendation
2018,-0.021048,-0.020159,-0.016946,0.016685,0.000674,-0.005936,-0.005936,-0.009049,-0.009049,-0.025946,-0.025946,recommendation
2019,-0.012845,-0.011956,-0.008744,0.024888,0.008877,0.002267,0.002267,-0.000846,-0.000846,-0.017744,-0.017744,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.413462,0.422697,0.272964,0.314505,-0.033207,0.277490,0.277490,0.246058,0.166389,0.358441,0.212891,recommendation
2097,0.416223,0.425581,0.275280,0.316415,-0.031623,0.279288,0.279288,0.249625,0.169405,0.358885,0.217627,recommendation
2098,0.418995,0.428444,0.277522,0.318317,-0.030008,0.281093,0.281093,0.253221,0.172382,0.359323,0.222414,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.061090,-0.060200,-0.056988,-0.023356,-0.039367,-0.045977,-0.045977,-0.049090,-0.049090,-0.065988,-0.065988,p05-p50
2016,-0.045630,-0.044741,-0.041528,-0.007896,-0.023907,-0.030517,-0.030517,-0.033631,-0.033631,-0.050528,-0.050528,p05-p50
2017,-0.031191,-0.030302,-0.027090,0.006542,-0.009469,-0.016079,-0.016079,-0.019192,-0.019192,-0.036090,-0.036090,p05-p50
2018,-0.021048,-0.020159,-0.016946,0.016685,0.000674,-0.005936,-0.005936,-0.009049,-0.009049,-0.025946,-0.025946,p05-p50
2019,-0.012845,-0.011956,-0.008744,0.024888,0.008877,0.002267,0.002267,-0.000846,-0.000846,-0.017744,-0.017744,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.413462,0.422697,0.272964,0.314505,-0.033207,0.277490,0.277490,0.246058,0.166389,0.358441,0.212891,p05-p50
2097,0.416223,0.425581,0.275280,0.316415,-0.031623,0.279288,0.279288,0.249625,0.169405,0.358885,0.217627,p05-p50
2098,0.418995,0.428444,0.277522,0.318317,-0.030008,0.281093,0.281093,0.253221,0.172382,0.359323,0.222414,p05-p50


...

scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.188996,-0.187313,-0.189651,-0.157241,-0.177964,-0.182292,-0.181948,-0.184219,-0.184219,-0.201801,-0.201424,p05-p50
2016,-0.147000,-0.145317,-0.147655,-0.115246,-0.135969,-0.140296,-0.139953,-0.142223,-0.142223,-0.159806,-0.159428,p05-p50
2017,-0.107181,-0.105498,-0.107836,-0.075427,-0.096150,-0.100477,-0.100134,-0.102404,-0.102404,-0.119987,-0.119609,p05-p50
2018,-0.070803,-0.069121,-0.071459,-0.039049,-0.059772,-0.064099,-0.063756,-0.066027,-0.066027,-0.083609,-0.083232,p05-p50
2019,-0.035857,-0.034175,-0.036513,-0.004103,-0.024826,-0.029153,-0.028810,-0.031081,-0.031081,-0.048663,-0.048286,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.140178,0.562271,1.718342,0.897791,2.969020,3.191958,2.488017,1.099400,2.197741,0.932684,3.908835,p05-p50
2097,0.134269,0.557583,1.727748,0.898525,3.014571,3.237244,2.527981,1.097377,2.214272,0.920251,3.960858,p05-p50
2098,0.128288,0.553023,1.736858,0.899226,3.060270,3.282652,2.568147,1.094910,2.230448,0.907945,4.012435,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.188996,-0.187313,-0.189651,-0.157241,-0.177964,-0.182292,-0.181948,-0.184219,-0.184219,-0.201801,-0.201424,p95-p50
2016,-0.147000,-0.145317,-0.147655,-0.115246,-0.135969,-0.140296,-0.139953,-0.142223,-0.142223,-0.159806,-0.159428,p95-p50
2017,-0.107181,-0.105498,-0.107836,-0.075427,-0.096150,-0.100477,-0.100134,-0.102404,-0.102404,-0.119987,-0.119609,p95-p50
2018,-0.070803,-0.069121,-0.071459,-0.039049,-0.059772,-0.064099,-0.063756,-0.066027,-0.066027,-0.083609,-0.083232,p95-p50
2019,-0.035857,-0.034175,-0.036513,-0.004103,-0.024826,-0.029153,-0.028810,-0.031081,-0.031081,-0.048663,-0.048286,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.140178,0.562271,1.718342,0.897791,2.969020,3.191958,2.488017,1.099400,2.197741,0.932684,3.908835,p95-p50
2097,0.134269,0.557583,1.727748,0.898525,3.014571,3.237244,2.527981,1.097377,2.214272,0.920251,3.960858,p95-p50
2098,0.128288,0.553023,1.736858,0.899226,3.060270,3.282652,2.568147,1.094910,2.230448,0.907945,4.012435,p95-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.079481,-0.077902,-0.078592,-0.042374,-0.065587,-0.069913,-0.069554,-0.073916,-0.073916,-0.089537,-0.089330,recommendation
2016,-0.059899,-0.058319,-0.059009,-0.022792,-0.046005,-0.050331,-0.049972,-0.054334,-0.054334,-0.069954,-0.069748,recommendation
2017,-0.042024,-0.040444,-0.041134,-0.004917,-0.028130,-0.032456,-0.032096,-0.036459,-0.036459,-0.052079,-0.051873,recommendation
2018,-0.027417,-0.025838,-0.026528,0.009690,-0.013524,-0.017850,-0.017490,-0.021852,-0.021852,-0.037473,-0.037266,recommendation
2019,-0.013977,-0.012398,-0.013087,0.023130,-0.000083,-0.004409,-0.004050,-0.008412,-0.008412,-0.024033,-0.023826,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.104894,0.127462,0.287187,0.236660,0.550622,0.783357,0.134170,0.415103,0.485772,0.081664,0.646431,recommendation
2097,0.105868,0.128058,0.288521,0.238859,0.558806,0.791568,0.137435,0.417640,0.487372,0.079889,0.648252,recommendation
2098,0.106920,0.128717,0.289841,0.241074,0.567002,0.799767,0.140763,0.420172,0.488952,0.078255,0.650043,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.079481,-0.077902,-0.078592,-0.042374,-0.065587,-0.069913,-0.069554,-0.073916,-0.073916,-0.089537,-0.089330,p05-p50
2016,-0.059899,-0.058319,-0.059009,-0.022792,-0.046005,-0.050331,-0.049972,-0.054334,-0.054334,-0.069954,-0.069748,p05-p50
2017,-0.042024,-0.040444,-0.041134,-0.004917,-0.028130,-0.032456,-0.032096,-0.036459,-0.036459,-0.052079,-0.051873,p05-p50
2018,-0.027417,-0.025838,-0.026528,0.009690,-0.013524,-0.017850,-0.017490,-0.021852,-0.021852,-0.037473,-0.037266,p05-p50
2019,-0.013977,-0.012398,-0.013087,0.023130,-0.000083,-0.004409,-0.004050,-0.008412,-0.008412,-0.024033,-0.023826,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.104894,0.127462,0.287187,0.236660,0.550622,0.783357,0.134170,0.415103,0.485772,0.081664,0.646431,p05-p50
2097,0.105868,0.128058,0.288521,0.238859,0.558806,0.791568,0.137435,0.417640,0.487372,0.079889,0.648252,p05-p50
2098,0.106920,0.128717,0.289841,0.241074,0.567002,0.799767,0.140763,0.420172,0.488952,0.078255,0.650043,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.079481,-0.077902,-0.078592,-0.042374,-0.065587,-0.069913,-0.069554,-0.073916,-0.073916,-0.089537,-0.089330,p95-p50
2016,-0.059899,-0.058319,-0.059009,-0.022792,-0.046005,-0.050331,-0.049972,-0.054334,-0.054334,-0.069954,-0.069748,p95-p50
2017,-0.042024,-0.040444,-0.041134,-0.004917,-0.028130,-0.032456,-0.032096,-0.036459,-0.036459,-0.052079,-0.051873,p95-p50
2018,-0.027417,-0.025838,-0.026528,0.009690,-0.013524,-0.017850,-0.017490,-0.021852,-0.021852,-0.037473,-0.037266,p95-p50
2019,-0.013977,-0.012398,-0.013087,0.023130,-0.000083,-0.004409,-0.004050,-0.008412,-0.008412,-0.024033,-0.023826,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.104894,0.127462,0.287187,0.236660,0.550622,0.783357,0.134170,0.415103,0.485772,0.081664,0.646431,p95-p50
2097,0.105868,0.128058,0.288521,0.238859,0.558806,0.791568,0.137435,0.417640,0.487372,0.079889,0.648252,p95-p50
2098,0.106920,0.128717,0.289841,0.241074,0.567002,0.799767,0.140763,0.420172,0.488952,0.078255,0.650043,p95-p50
